![CH3-ADS.png](./Media/CH3-ADS.png)

# <span style="color: rgb(204, 85, 0);">Gather Buffer Pool Consumers</span>

The purpose of this notebook is to query the sys.dm\_os\_buffer\_descriptors dynamic management view.  It returns information about all the data pages that are currently in the SQL Server buffer pool. The output of this view can be used to determine the distribution of database pages in the buffer pool according to database, object, or type. In SQL Server, this dynamic management view also returns information about the data pages in the buffer pool extension file. For more information, see Buffer Pool Extension, read this excellent doc on the [Buffer pool extension - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/buffer-pool-extension?view=sql-server-ver15)

When a data page is read from disk, the page is copied into the SQL Server buffer pool and cached for reuse. Each cached data page has one buffer descriptor. Buffer descriptors uniquely identify each data page that is currently cached in an instance of SQL Server. sys.dm\_os\_buffer\_descriptors returns cached pages for all user and system databases. This includes pages that are associated with the Resource database.

  

| Column name | Data type | Description |
| --- | --- | --- |
| database\_id | int | ID of database associated with the page in the buffer pool. Is nullable. |
| file\_id | int | ID of the file that stores the persisted image of the page. Is nullable. |
| page\_id | int | ID of the page within the file. Is nullable. |
| page\_level | int | Index level of the page. Is nullable. |
| allocation\_unit\_id | bigint | ID of the allocation unit of the page. This value can be used to join sys.allocation\_units. Is nullable. |
| page\_type | nvarchar(60) | Type of the page, such as: Data page or Index page. Is nullable. |
| row\_count | int | Number of rows on the page. Is nullable. |
| free\_space\_in\_bytes | int | Amount of available free space, in bytes, on the page. Is nullable. |
| is\_modified | bit | 1 = Page has been modified after it was read from the disk. Is nullable. |
| numa\_node | int | Nonuniform Memory Access node for the buffer. Is nullable. |
| read\_microsec | bigint | The actual time (in microseconds) required to read the page into the buffer. This number is reset when the buffer is reused. Is nullable. |
| is\_in\_bpool\_extension | bit | 1 = Page is in buffer pool extension. Is nullable. |
| pdw\_node\_id | int | Applies to: Azure Synapse Analytics, Analytics Platform System (PDW)<br><br>The identifier for the node that this distribution is on. |

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600)
DECLARE @sqlmajorver int

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

-- Note: in case of NUMA architecture, more than one entry per database is expected

SET @sqlcmd = 'SELECT ''Memory_checks'' AS [Category], ''Buffer_Pool_Consumers'' AS [Information], 
numa_node, COUNT_BIG(DISTINCT page_id)*8/1024 AS total_pages_MB, 
CASE database_id WHEN 32767 THEN ''ResourceDB'' ELSE DB_NAME(database_id) END AS database_name,
SUM(CONVERT(BIGINT,row_count))/COUNT_BIG(DISTINCT page_id) AS avg_row_count_per_page, 
SUM(CONVERT(BIGINT, free_space_in_bytes))/COUNT_BIG(DISTINCT page_id) AS avg_free_space_bytes_per_page
' + CASE WHEN @sqlmajorver >= 12 THEN ',is_in_bpool_extension' ELSE '' END + '
' + CASE WHEN @sqlmajorver = 10 THEN ',numa_node' ELSE '' END + '
' + CASE WHEN @sqlmajorver >= 11 THEN ',AVG(read_microsec) AS avg_read_microsec' ELSE '' END + '
FROM sys.dm_os_buffer_descriptors
--WHERE bd.page_type IN (''DATA_PAGE'', ''INDEX_PAGE'')
GROUP BY database_id' + CASE WHEN @sqlmajorver >= 10 THEN ', numa_node' ELSE '' END + CASE WHEN @sqlmajorver >= 12 THEN ', is_in_bpool_extension' ELSE '' END + '
ORDER BY total_pages_MB DESC;'
EXECUTE sp_executesql @sqlcmd;
